In [35]:
from glob import glob
from time import sleep
from os import path
from datetime import datetime
from dateutil.relativedelta import relativedelta
import googleapiclient.discovery
import json
import os

In [6]:
DEVELOPER_KEY = "KEY"
CHANNEL_YUTT = 'UCQJ6ESCWQotBwtJm0Ff_gyQ'
CHANNEL_SIRI = 'UCFVuMJgj2QWlg-6JZ1ye9qA'

In [7]:
def yt_search_list(channelId, **kwargs):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.search().list(
        part="snippet",
        channelId=channelId,
        order='date',
        maxResults=50,
        **kwargs
    )
    response = request.execute()

    return response

In [27]:
data = []
publishedBefore = datetime.utcnow()

# chunk by only requesting results for a year to prevent reaching the limit of max. returned pages
while True:  
    timeArgs = { 
        'publishedBefore': publishedBefore.isoformat('T')[:19] + 'Z',
        'publishedAfter': (publishedBefore - relativedelta(years=1) - relativedelta(days=1)).isoformat('T')[:19] + 'Z'
    }

    print(timeArgs)
    
    page = yt_search_list(CHANNEL_SIRI, **timeArgs)

    if page['pageInfo']['totalResults'] == 0:
        break

    data.append(page)

    while 'nextPageToken' in page:
        print('Getting page', page['nextPageToken'])
        
        page = yt_search_list(CHANNEL_YUTT, pageToken=page['nextPageToken'], **timeArgs)

        data.append(page)
        sleep(2)
    
    publishedBefore -= relativedelta(years=1)
    sleep(10)

{'publishedBefore': '2020-06-09T21:51:56Z', 'publishedAfter': '2019-06-08T21:51:56Z'}
{'publishedBefore': '2019-06-09T21:51:56Z', 'publishedAfter': '2018-06-08T21:51:56Z'}
{'publishedBefore': '2018-06-09T21:51:56Z', 'publishedAfter': '2017-06-08T21:51:56Z'}
Getting page CDIQAA
Getting page CGQQAA
{'publishedBefore': '2017-06-09T21:51:56Z', 'publishedAfter': '2016-06-08T21:51:56Z'}


In [28]:
def filter_items(data):
    """ Extract items and remove duplicates """
    seen = set()
    items = []

    for page in data:
        for item in page['items']:
            item_id = str(item['id'])

            if not item_id in seen:
                items.append(item)
                seen.add(item_id)
    
    return items

In [29]:
items = filter_items(data)

In [30]:
len(items)

152

In [31]:
# num of videos
sum([1 for p in items if p['id']['kind'] == 'youtube#video'])

113

In [32]:
# num of playlists
sum([1 for p in items if p['id']['kind'] == 'youtube#playlist'])

39

In [33]:
# num of channel
sum([1 for p in items if p['id']['kind'] == 'youtube#channel'])

0

In [34]:
#with open('search_list_siri.json', 'w') as f:
#    f.write(json.dumps(items))